<a href="https://colab.research.google.com/github/DomingosLemos/Deploy-cnn-test/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG16


# Import the necessary libraries

In [ ]:
!pip install split-folders
import splitfolders
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import h5py
#import gc
import tensorflow as tf
#from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
#from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

#models
from tensorflow.keras.applications.vgg16 import VGG16

### Mount the google drive where we have a zip with all the images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Unzip images to folders

In [ ]:
!7z x /content/drive/MyDrive/EDSA/images.7z -o/content/images


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/EDSA/                                      1 file, 4942431249 bytes (4714 MiB)

Extracting archive: /content/drive/MyDrive/EDSA/images.7z
--
Path = /content/drive/MyDrive/EDSA/images.7z
Type = 7z
Physical Size = 4942431249
Headers Size = 1206945
Method = LZMA2:24
Solid = +
Blocks = 3

  0%      0% 102          0% 114 - images/apple_pie/1057749.jpg                                         0% 185 - images/apple_pie/1295996.jpg                                         0% 186 - ima

### Split images with splitfolder
Put on:
* content/output/train with 70% (700 images for each folder)
* content/output/val with 20% (200 images for each folder)
* content/output/test with 10% (100 images for each folder)



In [ ]:
splitfolders.ratio('/content/images', output="output", seed=1337, ratio=(.7, .2, .1)) 

Copying files: 101000 files [01:54, 885.52 files/s] 


## VGG-16 model

As we have a small dataset for each category (700 images) we will use "Image Augmentation" to increase the size of our dataset a little

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator and get data. 
# Simpler augmentations were tested, with inferior results (unfortunately the results were lost and for the training time we didn't run again)

train_datagen_VGG = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40, 
                                   width_shift_range = 0.2, 
                                   height_shift_range = 0.2, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

# Validation data is not augmented
val_datagen_VGG = ImageDataGenerator( rescale = 1.0/255. )

train_dir = '/content/output/train'
validation_dir = '/content/output/val'

### Get the data

In [ ]:
# Flow training images in batches of 20 using train_datagen_VGG generator
train_generator = train_datagen_VGG.flow_from_directory(train_dir, batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

# Flow validation images in batches of 20 using val_datagen_VGG generator
validation_generator = val_datagen_VGG.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'categorical', target_size = (224, 224))



Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#Loading the Base Model

base_model = VGG16(input_shape = (224, 224, 3), # Standard shape for pre-trained model
                   include_top = False, # Leave out the last fully connected layer
                   weights = 'imagenet') #pre-training on ImageNet


58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
#Running a garbage collection process cleans up a huge amount of objects
gc.collect()

0

In [ ]:
#Since we don’t have to train all the layers, we make them non_trainable
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#Compile and Fit
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final softmax layer for classification
x = layers.Dense(101, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

# Compile model
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])


In [ ]:
# Early stop was defined based on previous experiments described on the net
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)

# Steps per epoch and total epochs were defined based on previous experiments described on the net
vgghist = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 1000, callbacks=[early_stop])  

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
100/100 [==============================] - 110s 1s/step - loss: 3.6548 - acc: 0.1875 - val_loss: 3.2545 - val_acc: 0.2528
Epoch 2/1000
100/100 [==============================] - 109s 1s/step - loss: 3.5385 - acc: 0.2170 - val_loss: 3.2161 - val_acc: 0.2598
Epoch 3/1000
100/100 [==============================] - 109s 1s/step - loss: 3.4909 - acc: 0.2195 - val_loss: 3.2393 - val_acc: 0.2560
Epoch 4/1000
100/100 [==============================] - 109s 1s/step - loss: 3.5377 - acc: 0.2115 - val_loss: 3.1902 - val_acc: 0.2642
Epoch 5/1000
100/100 [==============================] - 109s 1s/step - loss: 3.4744 - acc: 0.2045 - val_loss: 3.1636 - val_acc: 0.2688
Epoch 6/1000
100/100 [==============================] - 110s 1s/step - loss: 3.4472 - acc: 0.2100 - val_loss: 3.0688 - val_acc: 0.2779
Epoch 7/1000
100/100 [==============================] - 110s 1s/step - loss: 3.4744 - acc: 0.2145 - val_loss: 3.0982 - val_acc: 0.2783
Epoch 8/1000
100/100 [==============================] -

Note: Not being the first pre-trained model and giving such low values of acc and val_acc and so high of loss that we decided to abort

In [ ]:
model.save('food_vgg16_model.h5')